In [1]:
import json
import pprint
import csv
import pickle
import datetime
import sPickle

In [ ]:
#took ~40 minutes

a = datetime.datetime.now().replace(microsecond=0)

with open("/mnt/mypartition/pokitdok_project/npidata_20170108.jsonl", 'r') as f:
    the_data = [json.loads(l) for l in f]
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

In [3]:
#test to see that data is read in
len(the_data)

NameError: name 'the_data' is not defined

In [8]:
the_data[0]

{'degree': 'MD',
 'entity_type': 'individual',
 'enumeration_date': '2005-05-23T00:00:00',
 'fax': '3088652506',
 'first_name': 'DAVID',
 'full_name': 'DAVID A WIEBE',
 'gender': 'Male',
 'is_sole_proprietor': 'X',
 'last_name': 'WIEBE',
 'last_update_date': '2007-07-08T00:00:00',
 'licenses': [{'number': '12637', 'state': 'NE'}],
 'locations': [{'address_lines': ['Po Box 2168'],
   'city': 'Kearney',
   'country': 'US',
   'fax': '3088652506',
   'md5hash': '2f47fded67f1900b98146794d70fec25',
   'phone': '3088652512',
   'role': ['mailing'],
   'state': 'NE',
   'zipcode': '688482168'},
  {'address_lines': ['3500 Central Ave'],
   'city': 'Kearney',
   'country': 'US',
   'fax': '3088652506',
   'md5hash': 'f0384445e5a4ba49dfaa7acdd97c0632',
   'phone': '3088652512',
   'role': ['practice'],
   'state': 'NE',
   'zipcode': '688472944'}],
 'middle_name': 'A',
 'npi': '1679576722',
 'other_provider_identifiers': [{'id': '93420WI',
   'state': 'NE',
   'type': 'MEDICARE ID-TYPE UNSPECIFI

In [3]:
a = datetime.datetime.now().replace(microsecond=0)
#NOTE: regular pickle fails bc of memory error. use streaming-pickle.

#pickle.dump( the_data, open( "/mnt/mypartition/pokitdok_project/the_data.p", "wb" ) )
import sPickle

sPickle.s_dump( the_data, open( "/mnt/mypartition/pokitdok_project/the_data.spkl", "w" ) )

b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

MemoryError: 

In [6]:
#checking that all entries have npi's
#NOTE: don't run unless have more time bc takes foreverrrrr.....going to assume PokitDok has scrubbed those records without
#since they told us they did do some cleaning prior
a = datetime.datetime.now().replace(microsecond=0)

for i in the_data:
    try:
        if 'npi' in i:
            pass
        else:
            print("False")
    except KeyError:
        print(i)
        print("False")   
        
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

KeyboardInterrupt: 

In [ ]:
#load in 2nd file that we pickled 


In [40]:
#just to verify getting correct data
i = 0
for obj in data:
    if i<10:
        #this is the NPI
        print(data[i][0])
        #this is the med school
        print(data[i][9])
        #this is the grad year
        print(data[i][10])
    i=i+1

NPI
Medical school name
Graduation year
1821199704
PALMER COLLEGE CHIROPRACTIC -  WEST SUNNYVALE
1980
1487927612
LIFE CHIROPRACTIC COLLEGE - WEST
2010
1235146762
OTHER
1992
1285727842
OTHER
1981
1770581647
LIFE CHIROPRACTIC COLLEGE - WEST
1982
1447638408
OTHER
2014
1295821098
OHIO STATE UNIVERSITY COLLEGE OF MEDICINE
1984
1285616490
UNIVERSITY OF OKLAHOMA COLLEGE OF MEDICINE
1998
1063514289
OTHER
1997


### Make dictionary of npi's and med schools

In [7]:
#how many records have blank med schools?
#answer: none
dictMedSchools = {}
i=0
dups=0
for obj in data:
    #key is the npi, value is the med school
    if (dictMedSchools.get(obj[0]) is not None) & (dictMedSchools.get(obj[0]) != 'NULL'):
        dups=dups+1
    #accounting for duplicates and null values
    elif obj[9]=='':
        dictMedSchools[obj[0]] = 'NULL'
    else:
        dictMedSchools[obj[0]] = obj[9]
    i=i+1

print(i)
print(dups)

YES
YES
2111151
1151158


TODO: check whether duplicates are overwriting school names with 'Other' or nulls or not

In [9]:
len(dictMedSchools)

959993

In [11]:
#test on one record that should be in MedSchools dictionary
print(dictMedSchools.get('1114920261'))

OTHER


### Adding med school to datafile

In [27]:
for i in the_data:
    i['med_school'] = dictMedSchools.get(i['npi'])

In [37]:
#checking that worked
the_data[8]

{'degree': 'MD',
 'entity_type': 'individual',
 'enumeration_date': '2005-05-23T00:00:00',
 'fax': '8068922726',
 'first_name': 'ROBERT',
 'full_name': 'ROBERT BISBEE',
 'gender': 'Male',
 'is_sole_proprietor': 'N',
 'last_name': 'BISBEE',
 'last_update_date': '2012-01-13T00:00:00',
 'licenses': [{'number': 'J8461', 'state': 'TX'}],
 'locations': [{'address_lines': ['5219 City Bank Pkwy', 'Ste 35'],
   'city': 'Lubbock',
   'country': 'US',
   'fax': '8067222908',
   'md5hash': '25d62645fab36a478833c0f5031b6b85',
   'phone': '8067852045',
   'role': ['mailing'],
   'state': 'TX',
   'zipcode': '794073545'},
  {'address_lines': ['113 Walnut St'],
   'city': 'Idalou',
   'country': 'US',
   'fax': '8068922726',
   'md5hash': 'e06be513999cf34cffde1f54ce831e30',
   'phone': '8068922537',
   'role': ['practice'],
   'state': 'TX',
   'zipcode': '793294003'}],
 'med_school': 'TEXAS TECH UNIVERSITY HEALTH SCIENCE CENTER SCHOOL OF MEDICINE',
 'npi': '1750384806',
 'other_provider_identifiers':

### Now doing the same for the year of graduation

### Step 1: make dictionary of npi's and grad years

In [42]:
#how many records have blank grad years?
#answer: none
dictGradYears = {}
i=0
dups=0
for obj in data:
    #key is the npi, value is the grad year
    if (dictGradYears.get(obj[0]) is not None) & (dictGradYears.get(obj[0]) != 'NULL'):
        dups=dups+1
    #accounting for duplicates and null values
    elif obj[10]=='':
        dictGradYears[obj[0]] = 'NULL'
    else:
        dictGradYears[obj[0]] = obj[10]
    i=i+1

print(i)
print(dups)

2111151
1147907


In [44]:
print(len(dictGradYears))

959993


matching up?

In [ ]:
#test on one record that should be in MedSchools dictionary
print(dictGradYears.get('1114920261'))

### Step 2: add grad years to datafile by searching dictionary on NPI

In [ ]:
for i in the_data:
    i['grad_year'] = dictGradYears.get(i['npi'])

In [ ]:
#checking that worked
the_data[8]